In [2]:
%load_ext autoreload
%autoreload 2

# Analyse des accidents de la route en France  
## Introduction
Les accidents de la route constituent un enjeu majeur de santé publique en France, avec plusieurs milliers de morts et de blessés graves chaque année. Leur compréhension repose sur l’analyse conjointe de facteurs temporels, environnementaux, infrastructurels et humains.

Ce projet s’appuie sur les données officielles des accidents corporels de la circulation en France, mises à disposition par les autorités publiques. Ces données décrivent, pour chaque accident, ses caractéristiques temporelles, géographiques, environnementales ainsi que certaines informations sur les usagers et les véhicules impliqués.

L’objectif de ce travail est de proposer une analyse statistique exploratoire des accidents de la route, en mettant l’accent sur les facteurs associés à la **gravité des accidents**, tout en tenant compte des limites inhérentes aux données disponibles.


In [ ]:
!pip install -r requirements.txt

# modules 
import requests
import pandas as pd
import numpy as np
import re
import io
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D


# Fonctions
from modules import module_BAAC
from modules import graph_baac
from modules import module_region

### Présentation des données

L’analyse repose principalement sur les jeux de données des accidents de la route survenus en France de 2005 à 2024, ces données sont issues d'API publiques (data.gouv.fr, INSEE et transport.data.gouv.fr). Chaque ligne correspond à un accident et décrit notamment :

- la date et l’heure de l’accident, 
- la localisation géographique (coordonnées, département, commune), 
- les conditions environnementales (luminosité, conditions atmosphériques, état de la chaussée),
- le sexe du conducteur, 
- le nombre de véhicules et d’usagers impliqués, 
- la gravité de l’accident

Le jeux de données antérieurs à 2024 seront mobilisés ponctuellement pour étudier les évolutions temporelles à long terme. Chaque jeu de donnée est intitulé 'BAAC_table_année' et le jeu de donnée regroupant tous les accidents depuis 2005 est intitulé 'BAAC_table'. 


### Construction du dataframe principal

In [ ]:
# API Bases de données annuelles des accidents corporels de la circulation routière – 2005 à 2024 (gouvernement.fr)
BAAC_url = "https://www.data.gouv.fr/api/1/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/"

# URLs d'export CSV : consommation quotidienne d’alcool (Odissé - Santé Publique France)
URL_CONSO = "https://odisse.santepubliquefrance.fr/explore/dataset/alcool-consommation-quotidienne-region/download/?format=csv"

# URLs d'export CSV : alcoolisation ponctuelle importante (Odissé - Santé Publique France)
URL_API   = "https://odisse.santepubliquefrance.fr/explore/dataset/alcool-alcoolisation-ponctuelle-importante-api-mensuel-et-hebdomadaire-region/download/?format=csv"

# URLs d'export .XLSX : impoosition par région (gouvernement.fr)
url_impot = "https://www.impots.gouv.fr/node/25517"   # page with the file

In [5]:
BAAC_resources = module_BAAC.get_resources(BAAC_url)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = 300
BAAC_resources[["description", "url"]].head()

,description,url
0,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2024.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100210/2024.csv
1,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2023.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142813/2023.csv
2,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv
3,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv
4,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv


In [6]:
selected_BAAC_table = module_BAAC.select_baac_tables(BAAC_resources)
selected_BAAC_table.head(2)

,description,url,table,year
0,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv,caracteristiques,2024
1,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv,lieux,2024


In [7]:
BAAC_table = module_BAAC.build_baac_dataframe(selected_BAAC_table)
BAAC_table.head()

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,...,dep,year,id_vehicule_x,secu1,secu2,secu3,id_vehicule_y,motor,vma,id_usager
0,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200500000001,2,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Choix des variables conservées

La table principale BAAC_table comporte 61 variables, qui ne sont pas toutes pertinentes dans le cadre de cette étude. Nous avons décidé de conserver les variables suivantes : 'grav' (gravité de l'accident), 'year', 'mois', 'jour', 'hrmn' (→ heure), 'lat', 'long', 'dep', 'agg' (agglomération ou non), 'vma' (vitesse limite), 'nbv', 'catr', 'circ', 'lum', 'atm', 'surf' (paramètres physiques), 'sexe', 'an_nais' (année de naissance). Sur certaines tables, 'hrmn' est un string de la forme 'HH:MM' (avant 2019), sur d'autres 'hrmn' est de la fome HHMM, nous avons donc remplacé 'hmrn' par une variable 'hour' en conservant seulement l'heure de l'accident HH de type int. 


In [8]:
BAAC_table['hour'] = BAAC_table['hrmn'].apply(graph_baac.hrmn_to_hour)

variables_conservees = ["grav", "year", "mois", "jour", "hour", "lat", "long", "dep", "agg",
 "vma", "nbv", "catr", "circ", "lum", "atm", "surf", "sexe", "an_nais"]

BAAC_table = BAAC_table[variables_conservees]


In [9]:
BAAC_table.head()

,grav,year,mois,jour,hour,lat,long,dep,agg,vma,nbv,catr,circ,lum,atm,surf,sexe,an_nais
0,4,2005,1,12,19,5051500,0294400,590,2,NaN,02,3,2,3,1,1,1,1976
1,4,2005,1,12,19,5051500,0294400,590,2,NaN,02,3,2,3,1,1,1,1976
2,3,2005,1,12,19,5051500,0294400,590,2,NaN,02,3,2,3,1,1,2,1968
3,3,2005,1,12,19,5051500,0294400,590,2,NaN,02,3,2,3,1,1,2,1968
4,1,2005,1,12,19,5051500,0294400,590,2,NaN,02,3,2,3,1,1,1,1964


On crée ensuite les tables associées à chaque année à partir de la table principale : 

In [10]:
for i in range(2005, 2025):
    globals()[f'BAAC_table_{i}'] = BAAC_table.loc[BAAC_table['year'] == i].reset_index(drop=True)


### Valeurs manquantes
 
La table principale étant issue de la concaténation de vingt sous-tables, il est nécessaire de vérifier la proportion de mesures disponibles pour chaque variable et chaque année, afin d’éviter d’effectuer des analyses sur des valeurs manquantes (NaN).

Un tableau a été élaboré pour représenter, année par année, la proportion de valeurs effectivement renseignées pour chaque variable. Il permet par exemple de constater que que la variable 'vma' n’a été introduite qu’à partir de 2019. 

In [11]:
l = ["grav", "year", "mois", "jour", "hour", "lat", "long", "dep", "agg",
 "vma", "nbv", "catr", "circ", "lum", "atm", "surf", "sexe", "an_nais"]     #variables conservées

years = range(2005, 2025)
df_proportion = pd.DataFrame(index=years, columns=l)

for i in years:
    table = globals()[f'BAAC_table_{i}']
    for var in l:
        df_proportion.loc[i, var] = 1 - table[var].isna().sum() / len(table)

df_proportion = df_proportion.astype(float)

df_proportion = df_proportion.loc[:, (df_proportion != 1).any(axis=0)]

df_proportion

,lat,long,vma,nbv,catr,circ,atm,surf,an_nais
2005,0.323421,0.323421,0.0,1.000000,0.999995,1.000000,1.000000,1.000000,0.996738
2006,0.328526,0.328526,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.995026
2007,0.294926,0.294926,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.998316
2008,0.298939,0.298939,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.999736
2009,0.466751,0.466751,0.0,0.990449,1.000000,0.999217,0.999926,0.999095,0.999920
2010,0.466228,0.466228,0.0,0.998108,1.000000,0.999136,0.999986,0.999125,0.999632
2011,0.467052,0.467052,0.0,0.997440,1.000000,0.998481,0.999808,0.998268,0.999528
2012,0.464061,0.463985,0.0,0.996075,1.000000,0.996846,0.999954,0.995669,0.999913
2013,0.496620,0.496620,0.0,0.998394,1.000000,0.998992,0.999769,0.998695,0.999645
2014,0.519430,0.519430,0.0,0.997543,1.000000,0.998568,0.999763,0.997635,0.999863


Le choix des variables conservées est donc plutôt satisfaisant puisque les valeurs du tableau sont pour la plupart proches de 1. Il est enfin nécessaire de normaliser certaines variables dont le format peut changer au cours des années, comme par exemples les coordonnées géographiques dont le format change en 2019 comme on peut le constater ci-dessous. On créé donc les variables normalisées lat_norm et long_norm. On transforme aussi les str en int

In [12]:
for i in range(2005,2025) :
    table = globals()[f'BAAC_table_{i}'][['lat', 'long']].dropna()
    if not table.empty:
        print(i,':', table['lat'].iloc[0], table['long'].iloc[0])
    else:
        print(i,':', "No data")

2005 : 5051500 0294400
2006 : 4878219 0247355
2007 : 5051700 0291000
2008 : 4515400 -082600
2009 : 0 0
2010 : 0000000 0000000
2011 : 5053589 0295262
2012 : 5052928 0293643
2013 : 5051199 0290046
2014 : 0000000 0000000
2015 : 0000000 0000000
2016 : 0000000 0000000
2017 : 5051326 0292191
2018 : 5055737 0294992
2019 : 48,8962100 2,4701200
2020 : 48,7053500 2,4384100
2021 : 44,0389580000 4,3480220000
2022 : 44,5594200000 4,7257200000
2023 : 48,86638600 2,32347100
2024 : 47,56277000 6,75832000


In [13]:
BAAC_table = graph_baac.normalize_coordinates(BAAC_table)
BAAC_table.head(1)


,grav,year,mois,jour,hour,lat,long,dep,agg,vma,nbv,catr,circ,lum,atm,surf,sexe,an_nais
0,4,2005,1,12,19,50.515,0.2944,590,2,NaN,02,3,2,3,1,1,1,1976


In [14]:
BAAC_table = graph_baac.to_int(BAAC_table)
BAAC_table.head()

,grav,year,mois,jour,hour,lat,long,dep,agg,vma,nbv,catr,circ,lum,atm,surf,sexe,an_nais
0,4,2005,1,12,19,50.515,0.2944,590.0,2,NaN,2.0,3.0,2.0,3,1.0,1.0,1,1976.0
1,4,2005,1,12,19,50.515,0.2944,590.0,2,NaN,2.0,3.0,2.0,3,1.0,1.0,1,1976.0
2,3,2005,1,12,19,50.515,0.2944,590.0,2,NaN,2.0,3.0,2.0,3,1.0,1.0,2,1968.0
3,3,2005,1,12,19,50.515,0.2944,590.0,2,NaN,2.0,3.0,2.0,3,1.0,1.0,2,1968.0
4,1,2005,1,12,19,50.515,0.2944,590.0,2,NaN,2.0,3.0,2.0,3,1.0,1.0,1,1964.0


In [142]:
type(BAAC_table['grav'][0])

numpy.int64

## Problématique

Si la fréquence et la localisation des accidents de la route sont relativement bien documentées, la compréhension des **facteurs associés à la gravité des accidents** reste un enjeu central pour l’analyse et la prévention du risque routier.

La problématique principale de ce projet est donc la suivante :

> *Dans quelles conditions temporelles, environnementales et infrastructurelles les accidents de la route sont-ils les plus graves ?*

Cette question sera abordée à travers une analyse descriptive, exploratoire et multivariée des données, sans prétendre établir des relations causales strictes.


### Enjeux et choix méthodologiques

Plusieurs enjeux méthodologiques structurent ce travail :

Premièrement, les données sont observationnelles et ne permettent pas d’identifier des relations causales directes. De plus les variables territoriales agrégées (régions, départements) sont peu nombreuses au regard du nombre d’accidents observés, ce qui limite leur exploitation dans des modèles explicatifs.

En conséquence, l’analyse privilégie une approche descriptive pour dresser un état des lieux des accidents, une analyse exploratoire multivariée (ACP) pour comprendre la structure des relations entre variables ainsi que des modèles de régression visant à identifier les variables **associées** à la gravité des accidents.


## Organisation du notebook

Le notebook est structuré comme suit :

I. Analyse descriptive des accidents de la route en 2024  
II. Analyse des dynamiques temporelles à long terme (2005–2024)  
III. Analyse exploratoire multivariée (ACP)  
IV. Analyse des facteurs associés à la gravité des accidents  
V. Discussion des résultats et limites  
VI. Conclusion

Chaque partie combine des visualisations graphiques et des analyses statistiques adaptées aux objectifs poursuivis.


## I. Analyse descriptive des accidents de la route
### a) Dynamiques temporelles 

In [15]:
liste_df = [globals()[f'BAAC_table_{i}'] for i in range(2005, 2025)]
annees = [i for i in range(2005, 2025)]
occurences, occurences_graves, grav_moy, sexe_moy = graph_baac.dynamiques_temporelles(liste_df)

plt.plot(annees, sexe_moy)

plt.show()  

TypeError: Could not convert string '...' to numeric

In [153]:
len(BAAC_table_2024)

306824